# Cамоподобие Херста

In [52]:
import math

In [1]:
data = [
    (19, 1),
(20, 0),
(21, 2),
(22, 0),
(23, 2),
(24, 0),
(25, 1),
(26, 2),
(27, 1),
(28, 1),
(29, 3),
(30, 5),
(31, 5),
(32, 2),
(33, 3),
(34, 6),
(35, 5),
(36, 5),
(37, 2),
(38, 8),
(39, 4),
(40, 4),
(41, 10),
(42, 12),
(43, 8),
(44, 7),
(45, 7),
(46, 8),
(47, 2),
(48, 6),
(49, 2),
(50, 1),
(51, 4),
(52, 2),
(53, 1),
(54, 1)
]

In [55]:
block_sizes = [12, 9, 7, 6, 5, 4, 3, 2, 1]

In [53]:
def blocks(arr, size):
    blocks = []
    pos = 0
    for i in range(size, len(arr)+1, size):
        blocks.append(arr[pos:i])
        pos = pos + size
    return blocks

def expected_value(arr):
    """
        Математическое ожидание
    """
    return sum(arr)/len(arr)

def variance(arr, exp):
    """
        Дисперсия
    """
    d = [math.pow(x - exp, 2) for x in arr]
    return sum(d)/len(d)

In [65]:
dds = []
for block in blocks(data, 3):
    vals = [y for x, y in block]
    ev = expected_value(vals)
    v = variance(vals, ev)
    dds.append(v)
    
expected_value(dds)

2.685185185185185

In [51]:
expected_value([1, 2, 3])

2.0

In [54]:
variance([1, 2, 3], 4)

4.666666666666667

In [63]:
blocks(data, 4)

[[(19, 1), (20, 0), (21, 2), (22, 0)],
 [(23, 2), (24, 0), (25, 1), (26, 2)],
 [(27, 1), (28, 1), (29, 3), (30, 5)],
 [(31, 5), (32, 2), (33, 3), (34, 6)],
 [(35, 5), (36, 5), (37, 2), (38, 8)],
 [(39, 4), (40, 4), (41, 10), (42, 12)],
 [(43, 8), (44, 7), (45, 7), (46, 8)],
 [(47, 2), (48, 6), (49, 2), (50, 1)],
 [(51, 4), (52, 2), (53, 1), (54, 1)]]